# Test forward_model.py
### Mainly to test forward matching peaks and use the cleaned peaks for tomographic grain reconstruction
### Normally it is used as an extra step for the example notebook tomo_2_map.ipynb or tomo_2_map_minor_phase.ipynb
### Jan 2025

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import concurrent.futures

# %matplotlib ipympl

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.morphology import convex_hull_image

import ImageD11.columnfile
from ImageD11.grain import grain
from ImageD11.peakselect import select_ring_peaks_by_intensity
from ImageD11.sinograms.sinogram import GrainSinogram, build_slice_arrays, write_slice_recon, read_h5, write_h5, get_2d_peaks_from_4d_peaks
from ImageD11.sinograms.roi_iradon import run_iradon
from ImageD11.sinograms.tensor_map import TensorMap
import ImageD11.sinograms.dataset
import ImageD11.nbGui.nb_utils as utils

import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
from ImageD11.forward_model import forward_model

In [ ]:
# USER: Pass path to dataset file

dset_file = '/data/visitor/es1505/id11/20240926/PROCESSED_DATA/LGM_15/LGM_15_s3DXRD_inclusion1/LGM_15_s3DXRD_inclusion1_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)

ds.parfile = '/data/visitor/es1505/id11/20240926/SCRIPTS/HF/LGM_15_s3DXRD_inclusion1/pars.json' # specify the pars.json path

sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# load phases from parameter file

ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# pick a phase
phase_str = 'Pyrrhotite_hex2'

In [ ]:
# If the sinograms are only half-sinograms (we scanned dty across half the sample rather than the full sample), set the below to true:
is_half_scan = False

In [ ]:
if is_half_scan:
    ds.correct_bins_for_half_scan()

In [ ]:
# Import 4D peaks

cf_4d = ds.get_cf_4d_from_disk()

ds.update_colfile_pars(cf_4d, phase_name=phase_str)

print(f"Read {cf_4d.nrows} 4D peaks")

In [ ]:
# plot sino for cf_4d
forward_model.cf_plot_sino(cf_4d)

In [ ]:
# here we are filtering our peaks (cf_4d) to select only the strongest ones
# this time as opposed to indexing, our frac is slightly weaker but we are NOT filtering in dstar!!!!!
# this means many more peaks per grain = stronger sinograms

# USER: modify the "frac" parameter below and re-run the cell until the orange dot sits nicely on the "elbow" of the blue line
# this indicates the fractional intensity cutoff we will select
# if the blue line does not look elbow-shaped in the logscale plot, try changing the "doplot" parameter (the y scale of the logscale plot) until it does

cf_strong_frac = 0.999
cf_strong_dstol = 0.005

cf_strong = select_ring_peaks_by_intensity(cf_4d, frac=cf_strong_frac, dstol=cf_strong_dstol, dsmax=cf_4d.ds.max(), doplot=0.9)
print(cf_4d.nrows)
cf_strong.nrows

In [ ]:
# plot sino for cf_strong
forward_model.cf_plot_sino(cf_strong)

In [ ]:
# import the grains from disk

grains = ds.get_grains_from_disk(phase_str)
print(f"{len(grains)} grains imported")

In [ ]:
# assign peaks to the grains

peak_assign_tol = 0.25
utils.assign_peaks_to_grains(grains, cf_strong, peak_assign_tol)

for grain_label, g in enumerate(grains):
    g.npks_4d = np.sum(cf_strong.grain_id == grain_label)

In [ ]:
# let's make a GrainSinogram object for each grain

grainsinos = [GrainSinogram(g, ds) for g in grains]

In [ ]:
# Now let's determine the positions of each grain from the 4D peaks

for grain_label, gs in enumerate(grainsinos):
    gs.update_lab_position_from_peaks(cf_strong, grain_label)

In [ ]:
# get peaks for each grain, only to be able to see the peaks for individual grains
cf_grains = []
for i in range(len(grains)):
    cf_grains.append(forward_model.cf_filter_for_grain(cf_strong, grain_id = i))

In [ ]:
forward_model.cf_plot_sino(cf_grains)

In [ ]:
ucell = ds.phases.unitcells[phase_str]

In [ ]:
pars = ImageD11.parameters.read_par_file(ds.parfile)

# Testing the forward calculation and forward matching

In [ ]:
cf_matched_all, Comp_all = forward_model.forward_match_peaks(cf_strong, grains, ds, ucell, pars, ds_max=1.2, tol_angle=3, tol_pixel=5, thres_int=None)

In [ ]:
# plot all the matched peaks
# in this example, the scan was performed on a region-of-interest, so the number of 'useful' peaks have been filtered significantly from all peaks
forward_model.cf_plot_sino(cf_matched_all)

In [ ]:
# create cf_clean that is supposed to be matched with the indexed grains
cf_clean = forward_model.cf_remove_unmatched_peaks(cf_strong, cf_matched_all)

In [ ]:
# plot sino for cf_strong (before cleaning) and cf_clean (after cleaning)
forward_model.cf_plot_sino([cf_strong, cf_clean])

# Once cf_clean is obtained, one can continue with grain reconstruction with cf_clean instead of cf_strong
# See the rest of processing in the example notebook tomo_2_map.ipynb